# SPSUMA VERSION JIT

In [2]:
import math
import random
from numba import cuda,jit,njit
import numpy as np
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import time
import matplotlib.pyplot as plt   
import sys
import os
sys.path.append("/home/servelgar/tfg-cuda/vm/examples")
sys.path.append("/home/servelgar/tfg-cuda/vm/src")
sys.path.append("/home/servelgar/tfg-cuda/vm/tests")
print(sys.path)
from basicprobabilistic import basicprobabilistic
from spsuma import spsuma

['/home/servelgar/tfg-cuda/src', '/usr/lib64/python39.zip', '/usr/lib64/python3.9', '/usr/lib64/python3.9/lib-dynload', '', '/home/servelgar/.local/lib/python3.9/site-packages', '/usr/local/lib64/python3.9/site-packages', '/usr/local/lib/python3.9/site-packages', '/usr/lib64/python3.9/site-packages', '/usr/lib/python3.9/site-packages', '/home/servelgar/tfg-cuda/vm/examples', '/home/servelgar/tfg-cuda/vm/src', '/home/servelgar/tfg-cuda/vm/tests', '/home/servelgar/tfg-cuda/vm/examples', '/home/servelgar/tfg-cuda/vm/src', '/home/servelgar/tfg-cuda/vm/tests']


In [9]:
@njit
def spsuma(array_comp, initial_instructions):
    for i in range(1, array_comp.shape[0]):
        if(array_comp[i-1][0] != 0 and array_comp[i-1][1] != 0):
            for j in range(initial_instructions.shape[0]):   
                array_comp[i][j] = array_comp[i-1][j] - 1
                array_comp[i][2] = array_comp[i-1][2] + initial_instructions.shape[0] # sin pesos, voy a sumar la cantidad de instrucciones que se activen
                array_comp[i][3] = 2
        elif(array_comp[i-1][0] == 0 and array_comp[i-1][1] == 0):
            sol = array_comp[i-1][2]
            for j in range(array_comp.shape[0] - i):
                array_comp[j + i][2] = sol
                array_comp[j + i][3] = -1
        else:
            # mono instrucción por step:
            if (array_comp[i-1][0] == 0 and array_comp[i-1][1] != 0):
                array_comp[i][1] = array_comp[i-1][1] - 1
                array_comp[i][2] = array_comp[i-1][2] + 1
                array_comp[i][3] = 1
            
            elif(array_comp[i-1][1] == 0 and array_comp[i-1][0] != 0):
                array_comp[i][0] = array_comp[i-1][0] - 1
                array_comp[i][2] = array_comp[i-1][2] + 1
                array_comp[i][3] = 0
    
    return array_comp

In [10]:
def solucion_spsuma_cuda(array_comp):
    tam = array_comp.shape[0]
    s = f"sofparallel suma: {n_steps} steps\n"
    s += f"--STEP 0--\n"
    s += f"nº virus HOST 1: {n1}\n"
    s += f"nº virus HOST 2: {n2}\n"
    s += f"Actividades en paralelo: (i1, i2)\n"
    if tam <= 10:
        for i in range(1,tam):
            s += f"--STEP {i}--\n"
            if (array_comp[i][3] == 2):
                s += "Actividades en paralelo: (i1,i2)\n"
            elif (array_comp[i][3] == 1):
                s += "Actividad: i2\n"
            elif (array_comp[i][3] == 0):
                s += "Actividad: i1\n"
            else:
                s += "Actividad i3.\n"
                s += "FINAL DE LA SIMULACION\n"

            s += f"| HOST 1 | HOST 2 | HOST 3 |\n"
            s += f"|    {array_comp[i][0]}    |    {array_comp[i][1]}    |    {array_comp[i][2]}    |\n"
    else:
        for i in range(5):
            s += f"--STEP {i+1}--\n"
            if (array_comp[i][3] == 2):
                s += "Actividades en paralelo: (i1,i2)\n"
            elif (array_comp[i][3] == 1):
                s += "Actividad: i2\n"
            elif (array_comp[i][3] == 0):
                s += "Actividad: i1\n"
            else:
                s += "Actividad: i3.\n"
                s += "FINAL DE LA SIMULACION\n"

            s += f"| HOST 1 | HOST 2 | HOST 3 |\n"
            s += f"|    {array_comp[i][0]}    |    {array_comp[i][1]}    |    {array_comp[i][2]}    |\n"
            
        for i in range(tam-5,tam):
            if(i == (tam-5)):
                s += ".\n"
                s += ".\n"
                s += ".\n"
            s += f"--STEP {i+1}--\n"
            if (array_comp[i-1][3] == 2):
                s += "Actividades en paralelo: (i1,i2)\n"
            elif (array_comp[i-1][3] == 1):
                s += "Actividad: i2\n"
            elif (array_comp[i-1][3] == 0):
                s += "Actividad: i1\n"
            else:
                s += "Actividad: i3\n"
                s += "FINAL DE LA SIMULACION\n"

            s += f"| HOST 1 | HOST 2 | HOST 3 |\n"
            s += f"|     {array_comp[i-1][0]}    |     {array_comp[i-1][1]}    |    {array_comp[i-1][2]}    |\n"
                  
    return s

In [11]:
n1 = 4
n2 = 5
n_steps =9
initial_instructions = np.zeros((2,),dtype=np.int64) #i1 e i2 activas, que a su vez apuntan al host1 y host2 (posicion 0 y 1 del array de computacion)
array_comp = np.zeros((n_steps,4), dtype=np.int64)
array_comp[0,0] = n1
array_comp[0,1] = n2

array_comp = spsuma(array_comp, initial_instructions)
print(solucion_spsuma_cuda(array_comp))

sofparallel suma: 9 steps
--STEP 0--
nº virus HOST 1: 4
nº virus HOST 2: 5
Actividades en paralelo: (i1, i2)
--STEP 1--
Actividades en paralelo: (i1,i2)
| HOST 1 | HOST 2 | HOST 3 |
|    3    |    4    |    2    |
--STEP 2--
Actividades en paralelo: (i1,i2)
| HOST 1 | HOST 2 | HOST 3 |
|    2    |    3    |    4    |
--STEP 3--
Actividades en paralelo: (i1,i2)
| HOST 1 | HOST 2 | HOST 3 |
|    1    |    2    |    6    |
--STEP 4--
Actividades en paralelo: (i1,i2)
| HOST 1 | HOST 2 | HOST 3 |
|    0    |    1    |    8    |
--STEP 5--
Actividad: i2
| HOST 1 | HOST 2 | HOST 3 |
|    0    |    0    |    9    |
--STEP 6--
Actividad i3.
FINAL DE LA SIMULACION
| HOST 1 | HOST 2 | HOST 3 |
|    0    |    0    |    9    |
--STEP 7--
Actividad i3.
FINAL DE LA SIMULACION
| HOST 1 | HOST 2 | HOST 3 |
|    0    |    0    |    9    |
--STEP 8--
Actividad i3.
FINAL DE LA SIMULACION
| HOST 1 | HOST 2 | HOST 3 |
|    0    |    0    |    9    |

